# Data explorer for NWIS datat
NWIS is the National Water Information System, with data on surface and groundwater in the USA.
This notebook uses data delivered in a sort of standardized tab-delimited text format 'rdb'.  
Documentation for the format is at 
https://waterdata.usgs.gov/nwis/?tab_delimited_format_info  
and   
https://pubs.usgs.gov/of/2003/ofr03123/6.4rdb_format.pdf  
updates: https://help.waterdata.usgs.gov/news/June-10-2016  

Actual data (see the example URLs included below in one of the cells) shows some variability in conformance to the spec, and also probably reflect evolution of the spec over time.

Important pattern is header in the text file with lines beginning with '#' character; 

* Header:  
The header section contains zero or more lines with information such as disclaimers, sites, parameter and location names. 
The header comment lines start with a sharp sign (#) followed by (one or more...) space character followed by any text desired.

* Column-name row:  
The header is followed by exactly one tab-delimited column-name row. The fields in the tab-delimited column-name row contain the names of each column.

* Column definition row:  
This is followed by exactly one column-definition row.  The fields in the tab-delimited column-definition row contain the data definitions and optional column documentation for each column.

* Data section:  
Then is the data section consisting of any number of rows of tab-delimited data fields. Data rows must have exactly the same number of tab-delimited columns as both the column-name and column-definition rows. Null data values are allowed. 

NWIS data urls are like https://nwis.waterdata.usgs.gov/{state}/nwis/{dataseries}?site_no={siteIDstring}
the {state} part of the host name is optional, can be left out when site_no is provided; it uses standard two character state-name abbreviations (e.g. md, ca, nv...)
{dataseries} is a string identifying NWIS data categories. I haven't found a definitive list, but these are the ones I've figured out:
    * uv -- current data 
    * gwlevels -- groundwater levels
    * peak -- peak stream flow at gage
    * inventory -- site information
    * measurements -- streamflow data for site; https://help.waterdata.usgs.gov/output-formats#streamflow_measurement_data. data files do not include the variable names in header
    
This notebook takes an NWIS site_no as the URL parameter, and determines what data are available for the site from 


In [87]:
#imports
import pandas as pd
import numpy as np
import datetime as dt
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
import matplotlib.dates as mdates
import urllib.request
import json
#import urllib
import helpers
from datetime import datetime, date, time
from ipywidgets import widgets  
from IPython import get_ipython
from IPython.display import display
from matplotlib import pylab
from pylab import *
from IPython.display import clear_output
import warnings

# Utility functions

In [88]:
def is_number(s):
    try:
        float(s)
        return True
    except ValueError:
        return False

In [89]:
def getmetavalue(rdbdata, label,skipVal=-1):
    '''
    function to look for a text string in a text string
    input is a list of text string; 
    searches each string for string containing the 'label' string
    returns the following text, with '- ' and leading/trailing spaces removed
    .
    '''
    inlabel=''
    #first have to handle column names that are parameter numbers
    if label.split('_')[0].isnumeric():
        inlabel=label
        label = label.split('_')[0]

    
    rowcount=0
#    for line in rdbdata:
#        rowcount=rowcount+1
#    print('file lines: ' + str(rowcount))
#    return('nothing')
#    print('getmetadatavalue for ' + label)
    for line in rdbdata:
        strline = str(line)
        #print('line:' + strline)
        if (label in strline):
            out= strline.split(label)[1]
            out = out.strip()
            if (strline[1:2]=='- '):
                out = strline.split('- ')[1]
            if (len(inlabel) == 0):
                return(out)
            else:
                if (inlabel[-3:]=='_cd'):
                    return(out+' code')
                else:
                    return(out)
        rowcount=rowcount+1
        if (skipVal > 0 and rowcount>skipVal): 
            return('Field not documemented')
    return('Field not documemented')



# Prepare Data:

In [90]:
%%javascript
function getQueryStringValue (key){  
    return unescape(window.location.search.replace(new RegExp("^(?:.*[&\\?]" + escape(key).replace(/[\.\+\*]/g, "\\$&") + "(?:\\=([^&]*))?)?.*$", "i"), "$1"));
}
IPython.notebook.kernel.execute("dataurl='".concat(getQueryStringValue("dataurl")).concat("'"));

IPython.notebook.kernel.execute("site_no='".concat(getQueryStringValue("site_no")).concat("'"));
//get the URL parameters and pass to python

<IPython.core.display.Javascript object>

In [91]:
# Some example URLS included here for various NWIS data

if (len(dataurl)==0):
    print('No URL paramteter')
    # for testing without URL parameter
    
    # Water Quality Samples for the Nation
    #dataurl = 'https://nwis.waterdata.usgs.gov/usa/nwis/qwdata/?site_no=07373420&format=rdb'
    #dataurl = 'https://nwis.waterdata.usgs.gov/usa/nwis/qwdata/?site_no=07373420&pm_cd_compare=Greater%20than&format=rdb'
    #site_no='07373420'
    
    # groundwater levels
    #dataurl = 'http://nwis.waterdata.usgs.gov/pa/nwis/gwlevels?site_no=400650075514001&agency_cd=USGS&format=rdb'
    # note this gets same result (no '/pa' in URL):
    #dataurl = 'https://nwis.waterdata.usgs.gov/nwis/gwlevels?site_no=400650075514001&agency_cd=USGS&format=rdb'
    #site_no='400650075514001'
    
    # current conditions
    #dataurl = 'https://waterdata.usgs.gov/nwis/uv/?site_no=06191000&format=rdb'
    #site_no='06191000'
    
    #peak stream flow service
    dataurl = 'https://nwis.waterdata.usgs.gov/nwis/peak/?site_no=01063310&agency_cd=USGS&format=rdb'
    site_no='01063310'
    
    #site information:
    #dataurl = 'https://waterdata.usgs.gov/nwis/inventory/?site_no=06022500&format=rdb'
    #site_no='06022500'
    

    # site measurements, rdb expanded, works in most cases but doesn't always add more columns:
    #dataurl = 'https://waterdata.usgs.gov/nwis/measurements?site_no=08158810&format=rdb_expanded'
    #site_no='08158810'
    

else:
    dataurl= dataurl + '&agency_cd=USGS&format=rdb'

No URL paramteter


In [93]:
# Get the data

print(dataurl)
print('Downloading the dataset. Please wait...')

data = urllib.request.urlopen(dataurl) # it's a file like object and works just like a file
# counter for the number of comment lines at the top of the file
skipVal = 0
datalist=[]
for line in data:
    strline = str(line)
    strline = strline.replace("\\r",'')
    strline = strline.replace("\\n",'')
    strline = strline.replace("\'",'')
    #print('line:' + strline)
    if '#' in strline:
        if (strline.find(site_no)>-1):
            site_label=(strline.split('# ')[1].strip())
            #remove extra internal spaces
            site_label = " ".join(site_label.split())
        #print(strline.split('#')[1].strip())
        if (len(strline.split('#')[1].strip()) > 0): 
            out = strline.split('#')[1].strip()
            out = " ".join(out.split())
            #print('output to datalist: ' + out)
            datalist.append(out)
    else:
        break
    skipVal = skipVal + 1
print('Rows to skip: ' + str(skipVal))
global df
df = pd.read_csv(dataurl, sep='\t',skiprows=range(0, skipVal),encoding='utf-8')

#first row is formatting information, save for later
formats = df.head(1)
#print('formats: ' + str(formats))
#print(str(formats['agency_cd'][0]))
#drop the formatting information row
df = df.drop(df.index[0])
#print(df.info)
print('Ready')

https://nwis.waterdata.usgs.gov/nwis/peak/?site_no=01063310&agency_cd=USGS&format=rdb
Rows to skip: 64
Ready


In [94]:
# some data introspection

fielddescriptions={}

for col in df.columns:
    mdstring = getmetavalue(datalist, str(col))

    print('field: ' + str(col) + '. '+ mdstring + '. Format: ' + str(formats[col][0]) )
    if (col[-1:]=='s'):
        for item in df[col].tolist():
            if item.isnumeric():
                datatype='pinteger'
            elif: is_number(item):
                datatype='float'
            else:
                datatype='string'
                continue
    
    fielddescriptions[col] = {'desc':mdstring, 
                              'format':str(formats[col][0]),
                             'type':datatype}    
    
print(fielddescriptions['site_no']['format'])

field: agency_cd. Agency Code. Format: 5s
1     USGS
2     USGS
3     USGS
4     USGS
5     USGS
6     USGS
7     USGS
8     USGS
9     USGS
10    USGS
11    USGS
12    USGS
13    USGS
14    USGS
15    USGS
16    USGS
17    USGS
18    USGS
19    USGS
20    USGS
21    USGS
22    USGS
Name: agency_cd, dtype: object
field: site_no. USGS station number. Format: 15s
1     01063310
2     01063310
3     01063310
4     01063310
5     01063310
6     01063310
7     01063310
8     01063310
9     01063310
10    01063310
11    01063310
12    01063310
13    01063310
14    01063310
15    01063310
16    01063310
17    01063310
18    01063310
19    01063310
20    01063310
21    01063310
22    01063310
Name: site_no, dtype: object
field: peak_dt. Date of peak streamflow (format YYYY-MM-DD). Format: 10d
1     1996-04-17
2     1996-10-21
3     1998-06-14
4     1999-09-17
5     2000-03-28
6     2001-04-12
7     2002-05-14
8     2003-03-30
9     2004-04-02
10    2005-04-03
11    2005-10-15
12    2007-04-16


In [16]:
df.iloc[0]
pd.DataFrame(df.count()).transpose().style

#reformat strings to dates  TBD: look for column names ending in '_dt' and work on those
for col in df.columns:
    if (col[-3:] == '_dt'):
        print(col)
        df[col] =  pd.to_datetime(df[col], format='%Y-%m-%d', errors='ignore')

display("Number of values in each column",pd.DataFrame(df.count()).transpose().style)


'Number of values in each column'

,agency_cd,site_no,datetime,tz_cd,82651_00060,82651_00060_cd,82652_00065,82652_00065_cd
0,736,736,736,736,736,736,736,736


# Graph Data:

In [17]:
column_values = df.columns.values
print(df.columns.values)


#for index,value in enumerate(column_values):
#    print('index: ' + column_values[index])
#    column_values[index] = "%s %s"%(column_values[index], df['parm_nm'][df['parm_cd'] == value[1:]])
column_values = np.insert(column_values,0,'Choose Variable')
column_values

#select x axis
w = widgets.Dropdown(
    options=column_values,
    description='Number:',
    disabled=False,
    value = "Choose Variable"
)

#Describe what happens when the button changes its value
def on_change(change):
    #print('change: ' + str(change))
    if change['type'] == 'change' and change['name'] == 'value':
        try:
            df.sort_values('sample_dt', ascending=True)
            plt.plot(df['sample_dt'], df[change['new']])
            xlabel('Time(years)')
            title(change['new'])
        except ValueError:
            print('The column contains a Non-Integer Value! Nothing Can be Displayed!')


['agency_cd' 'site_no' 'datetime' 'tz_cd' '82651_00060' '82651_00060_cd'
 '82652_00065' '82652_00065_cd']


In [18]:
w.observe(on_change)
display(w)

Dropdown(description='Number:', options=('Choose Variable', 'agency_cd', 'site_no', 'datetime', 'tz_cd', '8265…

KeyError: 'sample_dt'

In [21]:
button = widgets.Button(description="Click Me!")
display(button)

def on_button_clicked(b):
    print("Button clicked.")

button.on_click(on_button_clicked)

Button(description='Click Me!', style=ButtonStyle())

Button clicked.
Button clicked.
Button clicked.


In [85]:
d = {'one' : pd.Series([1., 2., 3.], index=['a', 'b', 'c']),
    'two' : pd.Series([1., 2., 3., 4.], index=['a', 'b', 'c', 'd'])}

df = pd.DataFrame(d)

print(df)

print ("DF", type(df['one']), "\n", df['one'])

dfList = df['one'].tolist()

print ("DF list", dfList, type(dfList))

   one  two
a  1.0  1.0
b  2.0  2.0
c  3.0  3.0
d  NaN  4.0
DF <class 'pandas.core.series.Series'> 
 a    1.0
b    2.0
c    3.0
d    NaN
Name: one, dtype: float64
DF list [1.0, 2.0, 3.0, nan] <class 'list'>
